# Processing MCSL summer swimming result

In [282]:
from bs4 import BeautifulSoup
import csv
import os
import glob
import pandas as pd

In [283]:
#file/folder
htmlbasepath = "./data/2023/week1"
filename_html = 'OFvRH.html'
filename_output = filename_html.replace('.html','.csv')

htmlpath = os.path.join(htmlbasepath, f'{filename_html}')
# print(path)

In [284]:
with open(htmlpath, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')

# print(soup.prettify())

In [285]:
htmlfile_list = htmlbasepath.split('/')
year = htmlfile_list[2]
week = htmlfile_list[3]

In [286]:
table = soup.find("table")
cols = table.findAll("td")

content = []
result = []
record = []

for td in cols:
    try:
        string = ''.join(td.find(string=True))
        content.append(string)
    except: pass

event = content[0]

#loop the rest
for line in content:
    if line[0:5] == 'Event':
        event = line
    else:
        #start of new record, when it sees rank
        if (line[1:2] == '.' and len(line)==2) or (line[2:3] == '.' and len(line) == 3) or (line[2:3] == 'T' and len(line) == 4): #T is for tie
            if result:
                record.append(result)
                result = []
            result.append(year)
            result.append(week)
            result.append(event)
            result.append(line)
        else:
            result.append(line)



In [287]:
#take out relay record,
individual_record = []
for l in record:
    if 'Relay' not in l[2]: #third element on the list is where the event is located
        #print(l)
        individual_record.append(l)
        

In [288]:
# for rec in individual_record:
#     if len(rec) > 7:
#         print(rec)

In [289]:
df = pd.DataFrame(individual_record, columns =['year', 'week', 'event', 'rank', 'swimmer', 'seed', 'final']) 

In [290]:
df['swimmer']

0            Cohn, Naveh N (12) (OF)
1           Fry, Patrick M (12) (RH)
2            Fry, Daniel P (10) (RH)
3       Humphreys, Colby R (11) (OF)
4       Humphreys, Chase P (11) (OF)
                   ...              
330         Bingley, Hanna (17) (OF)
331       Owings, Hannah E (18) (RH)
332       Linkie, Sophie M (15) (OF)
333    Kleckner, Marissa J (18) (RH)
334        Costa, Sophia F (17) (RH)
Name: swimmer, Length: 335, dtype: object

In [291]:
#special handling for swimmer with () on its name
df['swimmer'] = df['swimmer'].apply(lambda x:x.replace('(Dan)', '- Dan'))
df['swimmer'] = df['swimmer'].apply(lambda x:x.replace('(Ben)', '- Ben'))
df['swimmer'] = df['swimmer'].apply(lambda x:x.replace('(Jojo)', '- Jojo'))
df['swimmer'] = df['swimmer'].apply(lambda x:x.replace('(Jorie)', '- Jorie'))




#splitting swimmer colunn into 3
df[['swimmer_name', 'swimmer_age', 'swimmer_team']] = df['swimmer'].str.split(pat='(', expand=True, n=2)

# #dropping progoma; swimmer column
# del df['swimmer']

In [292]:
# #replace function
# def replace_enhanced(x):
#     if x is None:
#         return(x)
#     else:
#         x.replace(')','')
#         return(x)
    
#remove ')' from age and team
df['swimmer_age'] = df['swimmer_age'].apply(lambda x:x.replace(')', ''))
df['swimmer_team'] = df['swimmer_team'].apply(lambda x:x.replace(')', ''))


In [293]:
# #add seconds for seed and final
# x = '34.88'
# x_list = x.split(':')

# time_list = x.split(':')
# seconds = float(time_list[0])*60 + float(time_list[1])
# print(seconds)


def x_to_seconds(x):
    x = x.replace('X', '') #replacing exibition time
    x_list = x.split(':')
    if x in ('NT', 'NS', 'DQ', 'DNF'):  # ignore NT, NS, DQ, and DNF
        return(x)
    elif len(x_list) == 1: #only seconds no minutes
         return(float(x_list[0]))
    else:
        return(float(x_list[0])*60 + float(x_list[1]))
# x_to_seconds(x)


df['seed_seconds'] = df['seed'].apply(lambda x:x_to_seconds(x))
df['final_seconds'] = df['final'].apply(lambda x:x_to_seconds(x))



In [294]:
#get event number
df['event_no'] = df['event'].apply(lambda x:x.split('-')[0].replace('Event ', ''))


In [295]:
df.head(2).T

,0,1
year,2021,2021
week,week5,week5
event,Event 1 - Boys 12U 100M Individual Medley,Event 1 - Boys 12U 100M Individual Medley
rank,1.,2.
swimmer,"Cohn, Naveh N (12) (OF)","Fry, Patrick M (12) (RH)"
seed,1:14.40,1:21.94
final,1:13.46,1:21.41
swimmer_name,"Cohn, Naveh N","Fry, Patrick M"
swimmer_age,12,12
swimmer_team,OF,RH


In [296]:
df.head()

,year,week,event,rank,swimmer,seed,final,swimmer_name,swimmer_age,swimmer_team,seed_seconds,final_seconds,event_no
0,2021,week5,Event 1 - Boys 12U 100M Individual Medley,1.,"Cohn, Naveh N (12) (OF)",1:14.40,1:13.46,"Cohn, Naveh N",12,OF,74.4,73.46,1
1,2021,week5,Event 1 - Boys 12U 100M Individual Medley,2.,"Fry, Patrick M (12) (RH)",1:21.94,1:21.41,"Fry, Patrick M",12,RH,81.94,81.41,1
2,2021,week5,Event 1 - Boys 12U 100M Individual Medley,3.,"Fry, Daniel P (10) (RH)",1:23.62,1:21.75,"Fry, Daniel P",10,RH,83.62,81.75,1
3,2021,week5,Event 1 - Boys 12U 100M Individual Medley,4.,"Humphreys, Colby R (11) (OF)",1:25.50,1:22.39,"Humphreys, Colby R",11,OF,85.5,82.39,1
4,2021,week5,Event 1 - Boys 12U 100M Individual Medley,5.,"Humphreys, Chase P (11) (OF)",1:24.18,1:22.40,"Humphreys, Chase P",11,OF,84.18,82.4,1


In [297]:
#export to csv with | delimited
df.to_csv(filename_output,sep='|', index=False) # Use pipe to seperate data


In [298]:
#counting swimmer by team/age
df.groupby(['swimmer_age', 'swimmer_team'])['swimmer_name'].count()

swimmer_age  swimmer_team
10           OF              19
             RH              23
11           OF              13
             RH              17
12           OF              23
             RH              18
13           OF              19
             RH              28
14           OF              17
             RH               8
15           OF              13
             RH               6
16           OF              11
             RH               4
17           OF               5
             RH               9
18           OF               7
             RH              17
7            OF              10
             RH              18
8            OF              20
             RH              11
9            OF              11
             RH               7
Jorie G      6 (RH            1
Name: swimmer_name, dtype: int64